# Denoising a phase-valued Signal

## This is example is adopted (different noise) and extended from Example 5.1 in
>Bergmann, R., Laus, F., Steidl, G. and Weinmann, A.,<br>
>Second order differences of cyclic data and applications in variational denoising.<br>
>SIAM Journal on Imaging Sciences, 7(4), pp. 2916–53, 2014<br>
>doi [10.1137/140969993](http://dx.doi.org/10.1137/140969993), arXiv [1405.5349](https://arxiv.org/abs/1405.5349)

This file is also available as standalone skript exporting `pdf`s and `csv`, see `src/examples/TV12_CPPA_SignalS1.jl`.

In [ ]:
using Manopt
using Plots
using LaTeXStrings

In [ ]:
x = 0:0.002:1
y = S1Signal.(x,true);
yR = S1Signal.(x,false);

In [ ]:
M = Circle()
yS = S1Point.(y)
yPow = PowPoint(yS)
ySn = addNoise.(Ref(M),yS,0.2);
yn = getValue.(ySn);
MPow = Power(M,size(yn))
ynPow = PowPoint(ySn);

In [ ]:
plot(x,yR,label="real",
    linetype=:scatter, marker=(2.5,stroke(0)),
    yticks=([-π,-π/2,0,π/2,π],[L"-\pi",L"-\frac{\pi}{2}",0,L"\frac{\pi}{2}",L"\pi"]),
    xticks=[0,1/4,1/2,3/4,1],
    legend=:topleft)
plot!(x,y,label="wrapped",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,yn,label="noisy",linetype=:scatter, marker=(2.5,stroke(0)))
meanSquaredError(MPow,yPow,ynPow)

In [ ]:
α = 0.75
proxMaps = [ (λ,x) -> proxDistance(MPow,λ,ynPow,x), (λ,x) -> proxTV(MPow,α*λ,x) ]
costF = (x) -> L2TV(MPow,ynPow,α,x);

In [ ]:
recTV = cyclicProximalPoint(MPow,costF,proxMaps,ynPow;
        debug = (d -> (d["Iteration"]%1000==1) ? print(d["Iteration"],"| λ=",d["λ"]," | last change ",distance(MPow,d["x"],d["xnew"]),"\n") : print("") ,
                Dict("λ"=>"","Iteration"=>0,"x"=>"","xnew"=>"") ,4),
        λ = i -> π/4/i
    );
meanSquaredError(MPow,yPow,recTV)

In [ ]:
plot(x,y,label="wrapped",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,yn,label="noisy",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,getValue.( getValue(recTV) ),label="rec. (TV)",linetype=:scatter, marker=(2.5,stroke(0)))

In [ ]:
β = 1
proxMaps2 = [ (λ,x) -> proxDistance(MPow,λ,yPow,x), (λ,x) -> proxTV2(MPow,β*λ,x) ]
costF2 = (x) -> L2TV2(MPow,yPow,β,x);

In [ ]:
recTV2 = cyclicProximalPoint(MPow,costF2,proxMaps2,ynPow;
        debug = (d -> (d["Iteration"]%1000==1) ? print(d["Iteration"],"| λ=",d["λ"]," | last change ",distance(MPow,d["x"],d["xnew"]),"\n") : print("") ,
                Dict("λ"=>"","Iteration"=>0,"x"=>"","xnew"=>"") ,4),
        λ = i -> π/4/i
    );
meanSquaredError(MPow,yPow,recTV2)

In [ ]:
plot(x,y,label="wrapped",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,yn,label="noisy",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,getValue.( getValue(recTV2) ),label="rec. (TV2)",linetype=:scatter, marker=(2.5,stroke(0)))

In [ ]:
α,β = 0.5,.5
proxMaps1p2 = [ (λ,x) -> proxDistance(MPow,λ,yPow,x), (λ,x) -> proxTV(MPow,α*λ,x), (λ,x) -> proxTV2(MPow,β*λ,x) ]
costF1p2 = (x) -> L2TVplusTV2(MPow,yPow,α,β,x);

In [ ]:
recTV1p2 = cyclicProximalPoint(MPow,costF1p2,proxMaps1p2,ynPow;
        debug = (d -> (d["Iteration"]%1000==1) ? print(d["Iteration"],"| λ=",d["λ"]," | last change ",distance(MPow,d["x"],d["xnew"]),"\n") : print("") ,
                Dict("λ"=>"","Iteration"=>0,"x"=>"","xnew"=>"") ,4),
        λ = i -> π/4/i
    );
meanSquaredError(MPow,yPow,recTV1p2)

In [ ]:
plot(x,y,label="wrapped",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,yn,label="noisy",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,getValue.( getValue(recTV1p2) ),label="rec. (TV1&2)",linetype=:scatter, marker=(2.5,stroke(0)))

In [ ]:
yRPow = PowPoint(RnPoint.(y))
ynRPow = PowPoint(RnPoint.(yn))
RPow = Power(Euclidean(1),size(yn))
α = 0.75
proxMapsR = [ (λ,x) -> proxDistance(RPow,λ,ynRPow,x), (λ,x) -> proxTV(RPow,α*λ,x) ]
costFR = (x) -> L2TV(RPow,ynRpow,α,x);
recTVR = cyclicProximalPoint(RPow,costFR,proxMapsR,ynRPow;
        debug = (d -> (d["Iteration"]%1000==1) ? print(d["Iteration"],"| λ=",d["λ"]," | last change ",distance(RPow,d["x"],d["xnew"]),"\n") : print("") ,
                Dict("λ"=>"","Iteration"=>0,"x"=>"","xnew"=>"") ,4),
        λ = i -> π/4/i
    );
plot(x,y,label="wrapped",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,yn,label="noisy",linetype=:scatter, marker=(2.5,stroke(0)))
plot!(x,getValue.( getValue(recTVR) ),label="reconstructed",linetype=:scatter, marker=(2.5,stroke(0)))